In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime
import seaborn as sns

In [ ]:
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

Regionalspending

In [ ]:
regional_spending = merged_data.groupby('Region')['TotalValue'].sum().reset_index()

Recency insight

In [ ]:
merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'])
last_transaction_date = merged_data.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_transaction_date['Recencydays'] = (merged_data['TransactionDate'].max() - last_transaction_date['TransactionDate']).dt.days

ATF

In [ ]:
transaction_counts = merged_data.groupby('CustomerID')['TransactionID'].nunique().reset_index()
average_frequency = transaction_counts['TransactionID'].mean()

In [ ]:
plt.bar(regional_spending['Region'], regional_spending['TotalValue'], color='blue')
plt.title('Regional spending')
plt.xlabel('Region')
plt.ylabel('Total value')
plt.show()

In [ ]:
plt.hist(last_transaction_date['Recencydays'], bins=15, color='green')
plt.title('Recency distribution')
plt.xlabel('Days since last transaction')
plt.ylabel('no. of customers')
plt.show()

In [ ]:
print("Average Transactions per Customer:", round(average_frequency, 2))

In [ ]:
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])
transactions['Quarter'] = transactions['TransactionDate'].dt.quarter

In [ ]:
merged = transactions.merge(products, on='ProductID', suffixes=('_transaction', '_product'))
quarterly_sales = merged.groupby('Quarter')['TotalValue'].sum().reset_index()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(quarterly_sales['Quarter'], quarterly_sales['TotalValue'])
plt.title('Quarterly Sales Trend')
plt.xlabel('Quarter')
plt.ylabel('Total Sales (USD)')
plt.grid(True)
plt.xticks(quarterly_sales['Quarter'])
plt.show()

In [ ]:
product_performance = merged.groupby('ProductName')['Quantity'].sum().round().astype(int).reset_index()
product_performance = product_performance.sort_values('Quantity', ascending=False)

In [ ]:
top_5 = product_performance.head(5)
bottom_5 = product_performance.tail(5)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.barplot(data=top_5, x='Quantity', y='ProductName')
plt.title('Top 5 Best-Selling Products')
plt.xlabel('Total Quantity Sold')

In [ ]:
plt.subplot(1, 2, 2)
sns.barplot(data=bottom_5, x='Quantity', y='ProductName')
plt.title('Bottom 5 Selling Products')
plt.xlabel('Total Quantity Sold')
plt.tight_layout()
plt.show()

In [ ]:
print("\nTop 5 Best-Selling Products:")
print(top_5.to_string())
print("\nBottom 5 Selling Products:")
print(bottom_5.to_string())
print("\nQuarterly Sales Summary:")
print(quarterly_sales.to_string())

In [ ]:
data = merged_data.copy()

ATV for churn risk scoring

In [ ]:
data['Avg_Transaction_Value'] = data['TotalValue'] / data['Quantity']
data['Avg_Transaction_Value'].fillna(0, inplace=True)

TOTal spenging for churn risk

In [ ]:
total_spending = data.groupby('CustomerID')['TotalValue'].sum().reset_index()
total_spending.columns = ['CustomerID', 'Total_Spending']
data = data.merge(total_spending, on='CustomerID', how='left')

Total Transactions per user for churn

In [ ]:
total_transactions = data.groupby('CustomerID')['TransactionID'].nunique().reset_index()
total_transactions.columns = ['CustomerID', 'Total_Transactions']
data = data.merge(total_transactions, on='CustomerID', how='left')

Customer Tenure (currentdate - Signupdate) for churn

In [ ]:
data['SignupDate'] = pd.to_datetime(data['SignupDate'])
current_date = datetime.now()
data['Customer_Tenure'] = (current_date - data['SignupDate']).dt.days

 Recency for churn

In [ ]:
last_transaction = data.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_transaction.columns = ['CustomerID', 'Last_Transaction_Date']
data = data.merge(last_transaction, on='CustomerID', how='left')
data['Recency'] = (current_date - data['Last_Transaction_Date']).dt.days

Churn risk calc.

In [ ]:
def min_max_scale(x):
    return (x - x.min()) / (x.max() - x.min())

In [ ]:
data['Recency_Score'] = min_max_scale(data['Recency'])
data['Frequency_Score'] = 1 - min_max_scale(data['Total_Transactions'])
data['Value_Score'] = 1 - min_max_scale(data['Total_Spending'])

In [ ]:
RECENCY_wt = 0.30
FREQUENCY_wt = 0.40
VALUE_wt = 0.30

In [ ]:
data['Churn_Risk_Indicator'] = (
    (data['Recency_Score'] * RECENCY_wt) +(data['Frequency_Score'] * FREQUENCY_wt) +(data['Value_Score'] * VALUE_wt)
)
data['Churn_Risk_Indicator'] = data['Churn_Risk_Indicator'].clip(0, 1)

In [ ]:
output_file = 'allmerged_with_churn.csv'
data.to_csv(output_file)
print(f"Dataset saved to somewhere '{output_file}'.")

In [ ]:
customer_risks = data[['CustomerID', 'Churn_Risk_Indicator']].drop_duplicates()
customer_risks = customer_risks.sort_values('CustomerID')

In [ ]:
plt.scatter(customer_risks['CustomerID'], customer_risks['Churn_Risk_Indicator'])
plt.xlabel('Customer ID')
plt.ylabel('Churn Risk Score')
plt.title('Churn Risk Score by Customer ID')
plt.show()